In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Project Objective

This project is an integrated document processing system that utilizes the capabilities of the Gemini API to perform three main tasks:

2. **Question and Answering (Q&A)** based on external documents using **RAG** architecture and the Chroma vector database.


In [4]:
from google import genai
from google.genai import types
from IPython.display import Markdown

genai.__version__

'0.2.2'

# Setting Up Your API Key

To use the services provided by external APIs, you need to set up your API key. Follow these steps:

1. **Obtain an API Key**: If you don't have an API key, you can get one from [AI Studio](https://aistudio.google.com/). Detailed instructions are available in the official documentation.

2. **Store the API Key in Kaggle Secrets**:
   - Go to the **Add-ons** menu in Kaggle.
   - Select **Secrets** and add your API key with the name `GOOGLE_API_KEY`.

3. **Use the API Key in Your Code**:
   - Once the key is stored, you can access it in your notebook using `os.environ["GOOGLE_API_KEY"]`.

> **Note**: Always keep your API key secure and avoid sharing it publicly.

In [8]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

### Large Language Models (LLMs) face two major limitations:

1. **Limited Knowledge**: They only "know" the information they were trained on.
2. **Limited Context Window**: They have a restricted input context size.

To address these limitations, the **Retrieval Augmented Generation (RAG)** technique is used. RAG consists of three key stages:

1. **Indexing**: Create a vector database to enable quick lookup of relevant information.
2. **Retrieval**: Retrieve documents related to the user's query.
3. **Generation**: Generate a natural language response using the retrieved information.

---
### Tools Used in This part:

- **Gemini API**: To create a vector database.
- **Chroma**: An open-source vector database for storing embeddings, embedding documents, and searching for relevant information.

> With Chroma, you can efficiently store, search, and retrieve documents to provide intelligent answers to user queries.

### Data

Here is a small set of documents you will use to create an embedding database.

In [10]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## Creating the Embedding Database with ChromaDB

In this step, you will create a vector database using **ChromaDB** to store and retrieve document embeddings. To generate embeddings, we will implement a custom function that utilizes the **Gemini API**.

## Key Concepts:
- **Documents**: These are the items stored in the database. They are inserted first and later retrieved based on their semantic similarity.
- **Queries**: These are the search terms or textual descriptions used to find relevant documents in the database.

## Implementation Details:
1. Use the `task_type` parameter as **retrieval_document** when generating embeddings for documents. This ensures that the embeddings are optimized for storage and retrieval.
2. Later, when querying the database, use **retrieval_query** for generating query embeddings.

In [14]:
#!pip install chromadb

In [21]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        self.document_mode = True  

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]


Now create a Chroma database client that uses the **GeminiEmbeddingFunction** and populate the database with the documents you defined above.

In [22]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

### Confirm that the data was inserted by looking at the database.

In [23]:
db.count()
# db.peek(1)

3

## **Retrieval**: Find relevant documents
To search the **Chroma database**, call the query method. Note that you also switch to the retrieval_query mode of embedding generation.

In [24]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "How do you use the touchscreen to play music?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

## Augmented Generation: Answer the Question

After retrieving relevant passages from a set of documents (the retrieval step), you can now construct a generation prompt to allow the Gemini API to generate a final answer.

In [25]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How do you use the touchscreen to play music?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



**Now use the generate_content method to to generate an answer to the question.**

In [26]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

To play music on your Googlecar, simply touch the "Music" icon on the large touchscreen display.


###  Congrats on building a Retrieval-Augmented Generation app!